In [1]:
# Clean the XML files
!python xmlconversion.py --verbose

images/Pica_pica_506.xml
images/Periparus_ater_936.xml
images/Turdus_merula747.xml
images/ErithacusRubecula0060.xml
images/Turdus_merula946.xml
images/Pica_pica_197.xml
images/Pica_pica_861.xml
images/Pica_pica_674.xml
images/2.xml
images/ErithacusRubecula1062.xml
images/Turdus_merula554.xml
images/Pica_pica_398.xml
images/ErithacusRubecula0099.xml
images/Pica_pica_271.xml
images/Periparus_ater_973.xml
images/Pica_pica_697.xml
images/Pica_pica_456.xml
images/ErithacusRubecula0219.xml
images/47.xml
images/Periparus_ater_985.xml
images/Periparus_ater_46.xml
images/ErithacusRubecula0290.xml
images/ErithacusRubecula0621.xml
images/ErithacusRubecula0759.xml
images/104.xml
images/Turdus_merula21.xml
images/ErithacusRubecula1044.xml
images/Turdus_merula735.xml
images/Pica_pica_39.xml
images/Pica_pica_799.xml
images/Pica_pica_875.xml
images/ErithacusRubecula0719.xml
images/Turdus_merula427.xml
images/Pica_pica_513.xml
images/Periparus_ater_592.xml
images/Periparus_ater_476.xml
images/Turdus_me

In [11]:
# Remove images with missing xml
!python remove_missing_xml.py

File Removed:  images/38.jpg
File Removed:  images/59.jpg
File Removed:  images/4.jpg
File Removed:  images/23.jpg
File Removed:  images/62.jpg
File Removed:  images/66.jpg
File Removed:  images/29.jpg
File Removed:  images/106.jpg
File Removed:  images/2.jpg
File Removed:  images/25.jpg
File Removed:  images/92.jpg
File Removed:  images/95.jpg
File Removed:  images/Turdus_merula967.jpg


In [ ]:
# Remove images tagged "No good" type in the console:
# for file in $(grep -l Nogood *); do     rm -i $file; done

0 files removed


In [13]:
# Train/Test Split of 90/10
!python partition_dataset.py -x -i ./images -r 0.1

In [21]:
# Export test and train to csv
!python xml_to_csv.py "images/"

/home/msc1/Desktop/Tensorflow-Object-Detection-API/Base/v1/object_detection/images/train
Successfully converted xml to csv.
/home/msc1/Desktop/Tensorflow-Object-Detection-API/Base/v1/object_detection/images/test
/home/msc1/Desktop/Tensorflow-Object-Detection-API/Base/v1/object_detection/images/test/Pica_pica_398.xml
/home/msc1/Desktop/Tensorflow-Object-Detection-API/Base/v1/object_detection/images/test/Periparus_ater_476.xml
/home/msc1/Desktop/Tensorflow-Object-Detection-API/Base/v1/object_detection/images/test/Periparus_ater_756.xml
/home/msc1/Desktop/Tensorflow-Object-Detection-API/Base/v1/object_detection/images/test/Periparus_ater_639.xml
/home/msc1/Desktop/Tensorflow-Object-Detection-API/Base/v1/object_detection/images/test/Turdus_merula23.xml
/home/msc1/Desktop/Tensorflow-Object-Detection-API/Base/v1/object_detection/images/test/Periparus_ater_772.xml
/home/msc1/Desktop/Tensorflow-Object-Detection-API/Base/v1/object_detection/images/test/Turdus_merula227.xml
/home/msc1/Desktop/Te

In [92]:
# Create the TF record

In [62]:
# Update the .PBTXT file
!code.exe './data/label_map.pbtxt'

/bin/bash: code.exe: command not found


In [6]:
# Create the TF Record (Train)
!python generate_tfrecord.py -x images/train -l data/label_map.pbtxt -o data/train.record

Successfully created the TFRecord file: data/train.record


In [7]:
# Create the TF Record (Test)
!python generate_tfrecord.py -x images/test -l data/label_map.pbtxt -o data/test.record

Successfully created the TFRecord file: data/test.record


In [2]:
# Setting the model path
PATH_TO_MODEL = "faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8"
# PATH_TO_MODEL = "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8"
# PATH_TO_MODEL = "efficientdet_d4_coco17_tpu-32"
# PATH_TO_MODEL = "centernet_hg104_1024x1024_coco17_tpu-32"

In [7]:
# Configure the config file
!code './training/TF2/training/'{PATH_TO_MODEL}'/pipeline.config'
# No of epochs 5800

## Hyperparameter and config file discussion

Each model will run for 5800 epochs or stopped early if noticable converence of classification and localization loss
is appearing (overfitting). 5800 epochs is a reletively small amount for today's standards. However, the dataset being used only contains roughly 4000 images and so improvment will decline rapidly before the model reaches close to 5800 epochs anyway. If a model does not appear to converge a discussion will take place in the evaluation as to whether the hyperparameters, epochs and steps will be adjusted for a re-execution.

Augmentation is included to randomly flip the images horizontally so that the model can be trained on slightly more diverse
images.

The images will be shuffled so that the model can be trained differenly each time it is run.

A batch size of 1 for all models is being used due to hardware limitations of the system.

i will be training 1024z1024 resolution models as the birds in the dataset are small species and I want to loose as little pixels as possible when scaling down any images.
### Resnet101 1024x1024
Parameters:

        - Backbone: resNet101
        - Resizer: 1024x1024
        - Number of classes: 90
        - Optimizer: momentum optimizer
      
        Feature Extractor:
            - type: 'faster_rcnn_resnet101_keras'
            - batch_norm_trainable: true
          
        - Post Processing score_converter: SOFTMAX
        - batch_size: 1
     
        Train Config:
            - batch_size: 1
            - sync_replicas: true
            - startup_delay_steps: 0
            - replicas_to_aggregate: 8
            - num_steps: 100000
          
###  SSD resnet152 1024x1024
Parameters:

        - Backbone: resNet152
        - Resizer: 1024x1024
        - Number of classes: 2
        - Optimizer: momentum optimizer
        
        - Post Processing score_converter: SIGMOID
        - batch_size: 1
         
        Train Config:
            - num_steps: 100000
            - startup_delay_steps: 0.0
            - replicas_to_aggregate: 8
            - max_number_of_boxes: 100
            - unpad_groundtruth_tensors: false
            - fine_tune_checkpoint_type: "detection"
            - use_bfloat16: true
            - fine_tune_checkpoint_version: V2
         
### Centernet hg104 1024x1024
Parameters:

        - Backbone: Hourglass-104
        - Resizer: 1024x1024
        - Number of classes: 90
        - Optimizer: adam optimizer

        - batch_size: 1

        Train Config:
            - num_steps: 50000
            - max_number_of_boxes: 100
            - unpad_groundtruth_tensors: false
            - fine_tune_checkpoint_type: "detection"
            - fine_tune_checkpoint_version: V2
             
             
More hyperparameters would have been added for a more diverse dataset to train on however. Due to hardware limitations the graphics card can only process a limited number before the process speed decreases rapidly.
             

In [5]:
!python model_main_tf2.py --model_dir=training/TF2/training/{PATH_TO_MODEL} --pipeline_config_path=training/TF2/training/{PATH_TO_MODEL}/pipeline.config --num_train_steps=5800 --alsologtostderr

2022-12-27 20:03:35.007397: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-12-27 20:03:36.257584: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-12-27 20:03:36.280553: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-27 20:03:36.280695: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-12-27 20:03:36.280710: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-12-27 20:03:36.282227: I tensorflow/stream_executor/

In [3]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./training/TF2/training/{PATH_TO_MODEL}/pipeline.config --trained_checkpoint_dir ./training/TF2/training/{PATH_TO_MODEL}/ --output_directory ./training/TF2/training/{PATH_TO_MODEL}/saved_model/

2022-12-27 21:43:27.546418: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-12-27 21:43:28.689176: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-12-27 21:43:28.715878: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-27 21:43:28.716442: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-12-27 21:43:28.716461: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-12-27 21:43:28.718237: I tensorflow/stream_executor/

In [5]:
# Exporting a Trained Inference Graph
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./training/TF2/training/{PATH_TO_MODEL}/pipeline.config --trained_checkpoint_dir ./training/TF2/training/{PATH_TO_MODEL}/ --output_directory ./training/TF2/training/{PATH_TO_MODEL}/saved_model/

2022-12-20 21:46:43.875703: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-12-20 21:46:45.241904: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-12-20 21:46:45.263959: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-12-20 21:46:45.263982: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: CMPR01
2022-12-20 21:46:45.263990: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: CMPR01
2022-12-20 21:46:45.264046: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2022-12-20 21:46:45.264066: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.91.3
2022-12-20 2